In [10]:
import time
from typing import Dict

import cv2
import numpy as np
from tabulate import tabulate

from pystream import Pipeline, Stage

In [11]:
# Cycle period for the input data
INPUT_PERIOD = 0.2
# Time to run the pipeline
ON_TIME = 5

In [12]:
class DummyStage(Stage):
    """A dummy stage that performs some convolutions to the input 2D array,
    and count how many input it has processed.
    
    It is recommended to define 'name' property in the stage instance init
    """

    def __init__(self, name: str) -> None:
        """Initialize stage

        Args:
            name (str): stage name
        """        
        self.count = 0
        self.name = name
        self.kernel = np.random.randint(-10, 10, size=(5, 5))

    def __call__(self, data: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
        """Main data processing of the stage, a mandatory method of the Stage 
        class. Note that the stage does not return a new dict object, but only
        modifies the input object and returns it as the output

        Args:
            data (Dict[str, np.ndarray]): the input array, packed
                in a dictionary

        Returns:
            Dict[str, np.ndarray]: the output array, packed
            in a dictionary
        """
        img = data["data"]
        for _ in range(100):
            img = cv2.filter2D(src=img, ddepth=-1, kernel=self.kernel)
        data["data"] = img
        self.count += 1
        return data

    def cleanup(self):
        """Cleanup method, called at the end of the pipeline.
        in this example, we want to reset the counter to 0"""
        self.count = 0

In [13]:
def generate_data() -> Dict[str, np.ndarray]:
    img = np.random.randint(0, 255, size=(480, 720, 3), dtype=np.uint8)
    return {"data": img}

def create_pipeline() -> Pipeline:
    # First, create the pipeline instance, we want to use the profiler
    pipeline = Pipeline(input_generator=generate_data, use_profiler=True)
    # Now, add 5 dummy stages to the pipeline.
    pipeline.add(DummyStage("Stage1"))
    pipeline.add(DummyStage("Stage2"))
    pipeline.add(DummyStage("Stage3"))
    pipeline.add(DummyStage("Stage4"))
    pipeline.add(DummyStage("Stage5"))
    return pipeline

def print_profile(latency: Dict[str, float], throughput: Dict[str, float]):
    data = []
    for k in latency.keys():
        d = [k, latency[k], throughput[k]]
        data.append(d)
    table = tabulate(data, headers=["Stage", "Latency (s)", "Throughput (d/s)"])
    print(table)

In [15]:
pipeline = create_pipeline()
print("Starting pipeline in serial...")
pipeline.serialize()
print(f"Streaming data each {INPUT_PERIOD} s...")
pipeline.start_loop(INPUT_PERIOD)
print(f"Waiting for {ON_TIME} s...")
time.sleep(ON_TIME)
print("Stopping pipeline...")
pipeline.stop_loop()

# Let's try read the last result and do cleanup
latest = pipeline.get_results()
print()
print("Last output shape:")
print(latest["data"].shape)
pipeline.cleanup()

# Get the profile
latency, throughput = pipeline.get_profiles()
print()
print("Pipeline profile:")
print_profile(latency, throughput)

Starting pipeline in serial...
Waiting for 5 s...
Stopping pipeline...

Last output shape:
(480, 720, 3)

Pipeline profile:
Stage       Latency    Throughput
--------  ---------  ------------
Stage1     0.173414       1.1411
Stage2     0.165654       1.1476
Stage3     0.180477       1.15147
Stage4     0.160584       1.14935
Stage5     0.162539       1.14617
Pipeline   0.842698       1.14617


In [17]:
pipeline = create_pipeline()
print("Starting pipeline in parallel...")
pipeline.parallelize()
print(f"Streaming data each {INPUT_PERIOD} s...")
pipeline.start_loop(INPUT_PERIOD)
print(f"Waiting for {ON_TIME} s...")
time.sleep(ON_TIME)
print("Stopping pipeline...")
pipeline.stop_loop()

# Let's try read the last result and do cleanup
latest = pipeline.get_results()
print()
print("Last output shape:")
print(latest["data"].shape)
pipeline.cleanup()

# Get the profile
latency, throughput = pipeline.get_profiles()
print()
print("Pipeline profile:")
print_profile(latency, throughput)

[PyStream] 2023-03-30 20:40:19,247 - INFO      : (Stage1 9496) Thread started...
[PyStream] 2023-03-30 20:40:19,365 - INFO      : (Stage2 15652) Thread started...
[PyStream] 2023-03-30 20:40:19,368 - INFO      : (Stage3 10060) Thread started...
[PyStream] 2023-03-30 20:40:19,368 - INFO      : (Stage4 16652) Thread started...
[PyStream] 2023-03-30 20:40:19,372 - INFO      : (Stage5 18200) Thread started...
[PyStream] 2023-03-30 20:40:19,372 - INFO      : (FinalStage 11460) Thread started...


Starting pipeline in parallel...
Streaming data each 0.2 s...
Waiting for 5 s...


[PyStream] 2023-03-30 20:40:24,432 - INFO      : (Stage5 18200) Terminating thread...
[PyStream] 2023-03-30 20:40:24,443 - INFO      : (FinalStage 11460) Terminating thread...


Stopping pipeline...

Last output shape:
(480, 720, 3)


[PyStream] 2023-03-30 20:40:24,620 - INFO      : (Stage3 10060) Terminating thread...
[PyStream] 2023-03-30 20:40:24,626 - INFO      : (Stage4 16652) Terminating thread...
[PyStream] 2023-03-30 20:40:24,638 - INFO      : (Stage2 15652) Terminating thread...
[PyStream] 2023-03-30 20:40:24,639 - INFO      : (Stage1 9496) Terminating thread...
[PyStream] 2023-03-30 20:40:25,446 - INFO      : (Stage5 18200) Thread terminated...
[PyStream] 2023-03-30 20:40:25,446 - INFO      : (FinalStage 11460) Thread terminated...
[PyStream] 2023-03-30 20:40:25,632 - INFO      : (Stage4 16652) Thread terminated...
[PyStream] 2023-03-30 20:40:25,632 - INFO      : (Stage3 10060) Thread terminated...
[PyStream] 2023-03-30 20:40:25,648 - INFO      : (Stage1 9496) Thread terminated...
[PyStream] 2023-03-30 20:40:25,648 - INFO      : (Stage2 15652) Thread terminated...



Pipeline profile:
Stage       Latency    Throughput
--------  ---------  ------------
Stage1     0.229076       4.34578
Stage2     0.220657       4.29952
Stage3     0.222065       4.2608
Stage4     0.233113       4.24435
Stage5     0.235186       4.21391
Pipeline   1.32022        4.21369
